# Using an LLM to interact with Pandas

It is possible to use an LLM to interact with a Pandas dataframe, something that can be useful when encountering a large, new dataset with unfamiliar content. Basically, the scenario can be thought of as a combination of a simple [RAG][1] system combined with [tool-calling][2]. As the tool-calls are fairly complex, we'll make use of the [llamaindex][3] framework to do the [heavy lifting][4].

**Caveat**: This is a bleeding edge technique and thus error prone, _and_ it uses python's `eval` function to execute code written by the LLM on your behalf which is a potential security risk. **Consider yourself warned.**

[1]: ../RAG-tutorial/intro.ipynb
[2]: ../LLM-tool-calling/LLM-tool-calling.ipynb
[3]: https://www.llamaindex.ai
[4]: https://docs.llamaindex.ai/en/stable/api_reference/query_engine/pandas/

The figure below outlines the process, compare it to the figure in the [RAG tutorial][1].

![Query pipeline](img/pipeline.png)

FIXME: Walkthrough

[1]: ../RAG-tutorial/intro.ipynb

## Install prerequisites

In [ ]:
!pip -q install ollama llama-index llama-index-experimental  llama-index-llms-ollama

## A simple Pandas agent

We will use the `titanic` dataset in `data/titanic.csv`, so import Pandas and load the dataset:

In [ ]:
import pandas as pd

df = pd.read_csv("data/titanic.csv")

In [ ]:
df.info()

In [ ]:
df.head()

Let's first try a "vanilla" call with a simple question about the dataset:

In [ ]:
OLLAMA_HOST = 'http://10.129.20.4:9090'
OLLAMA_MODEL = 'qwen2.5-coder:latest' # 'deepseek-coder-v2:latest' # 'deepseek-r1:70b' # 'llama3.3:latest'

In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from llama_index.experimental.query_engine import PandasQueryEngine

Settings.llm = Ollama(model=OLLAMA_MODEL, base_url=OLLAMA_HOST)
query_engine = PandasQueryEngine(df=df, verbose=False)
response = query_engine.query(
    "What is the key for the column outlining survial?",
)

In [ ]:
print(response)

OK, hopefully you got "Survived" as response in accordance with the output from `df.info()` above.

## Query pipelines

Let's dig into what just happened in detail by building our own query pipeline from scratch.

### Prompt templates

We can get the prompt templates used from the query_engine:

In [ ]:
prompts = query_engine.get_prompts()
for key in prompts.keys():
    print(f"--- {key} ---\n")
    print(prompts[key].template)
    print()

We could make up our own templates, but for now we'll us them as-is.

In [ ]:
pandas_prompt_str = (
    "You are working with a pandas dataframe in Python.\n"
    "The name of the dataframe is `df`.\n"
    "This is the result of `print(df.head())`:\n"
    "{df_str}\n\n"
    "Follow these instructions:\n"
    "{instruction_str}\n"
    "Query: {query_str}\n\n"
    "Expression:"
)

response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n\n"
    "Pandas Instructions (optional):\n{pandas_instructions}\n\n"
    "Pandas Output: {pandas_output}\n\n"
    "Response: "
)

The default `instruction_str` is:

In [ ]:
instruction_str = (
    "1. Convert the query to executable Python code using Pandas.\n"
    "2. The final line of code should be a Python expression that can be called with the `eval()` function.\n"
    "3. The code should represent a solution to the query.\n"
    "4. PRINT ONLY THE EXPRESSION.\n"
    "5. Do not quote the expression.\n"
)

## Build Query Pipeline

<!-- Looks like this: input query_str -> pandas_prompt -> llm1 -> pandas_output_parser -> response_synthesis_prompt -> llm2

Additional connections to response_synthesis_prompt: llm1 -> pandas_instructions, and pandas_output_parser -> pandas_output. -->

In [ ]:

from llama_index.core.query_pipeline import (QueryPipeline as QP, Link, InputComponent)
from llama_index.experimental.query_engine.pandas import PandasInstructionParser

pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str, df_str=df.head(5)
)

response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)

pandas_output_parser = PandasInstructionParser(df)

query_pipeline = QP(
    modules={
        "input": InputComponent(),
        "pandas_prompt": pandas_prompt,
        "llm1": llm,
        "pandas_output_parser": pandas_output_parser,
        "response_synthesis_prompt": response_synthesis_prompt,
        "llm2": llm,
    },
    verbose=True,
)

query_pipeline.add_chain(["input", "pandas_prompt", "llm1", "pandas_output_parser"])

query_pipeline.add_links(
    [
        Link("input", "response_synthesis_prompt", dest_key="query_str"),
        Link(
            "llm1", "response_synthesis_prompt", dest_key="pandas_instructions"
        ),
        Link(
            "pandas_output_parser",
            "response_synthesis_prompt",
            dest_key="pandas_output",
        ),
    ]
)

# add link from response synthesis prompt to llm2
query_pipeline.add_link("response_synthesis_prompt", "llm2")

In [ ]:
response = query_pipeline.run(
    query_str="What is the key for the column outlining survial?",
)

In [ ]:
from llama_index.core.query_pipeline import (QueryPipeline as QP, Link, InputComponent)
from llama_index.experimental.query_engine.pandas import PandasInstructionParser
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from llama_index.core import PromptTemplate


Settings.llm = Ollama(model=OLLAMA_MODEL, base_url=OLLAMA_HOST)

## Instructions (prompts)

In [ ]:
instruction_str = (
    "1. Convert the query to executable Python code using Pandas.\n"
    "2. The final line of code should be a Python expression that can be called with the `eval()` function.\n"
    "3. The code should represent a solution to the query.\n"
    "4. PRINT ONLY THE EXPRESSION.\n"
    "5. Do not quote the expression.\n"
)

pandas_prompt_str = (
    "You are working with a pandas dataframe in Python.\n"
    "The name of the dataframe is `df`.\n"
    "This is the result of `print(df.head())`:\n"
    "{df_str}\n\n"
    "Follow these instructions:\n"
    "{instruction_str}\n"
    "Query: {query_str}\n\n"
    "Expression:"
)

response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n\n"
    "Pandas Instructions (optional):\n{pandas_instructions}\n\n"
    "Pandas Output: {pandas_output}\n\n"
    "Response: "
)

pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str, df_str=df.head(5)
)
pandas_output_parser = PandasInstructionParser(df)
response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)
# llm = OpenAI(model="gpt-3.5-turbo")
llm = Ollama(model=OLLAMA_MODEL, base_url=OLLAMA_HOST)

In [ ]:
print(pandas_prompt)

## Build Query Pipeline

Looks like this: input query_str -> pandas_prompt -> llm1 -> pandas_output_parser -> response_synthesis_prompt -> llm2

Additional connections to response_synthesis_prompt: llm1 -> pandas_instructions, and pandas_output_parser -> pandas_output.

![Query pipeline](img/pipeline.png)

In [ ]:
qp = QP(
    modules={
        "input": InputComponent(),
        "pandas_prompt": pandas_prompt,
        "llm1": llm,
        "pandas_output_parser": pandas_output_parser,
        "response_synthesis_prompt": response_synthesis_prompt,
        "llm2": llm,
    },
    verbose=True,
)
qp.add_chain(["input", "pandas_prompt", "llm1", "pandas_output_parser"])
qp.add_links(
    [
        Link("input", "response_synthesis_prompt", dest_key="query_str"),
        Link(
            "llm1", "response_synthesis_prompt", dest_key="pandas_instructions"
        ),
        Link(
            "pandas_output_parser",
            "response_synthesis_prompt",
            dest_key="pandas_output",
        ),
    ]
)
# add link from response synthesis prompt to llm2
qp.add_link("response_synthesis_prompt", "llm2")

In [ ]:
response = qp.run(
    query_str="What is the correlation between survival and age?",
)

In [ ]:
print(response.message.content)

In [ ]:
print(df.head())

In [ ]:
response = qp.run(
    query_str="What are the keys of the columns?",
)

In [ ]:
print(response.message.content)

In [ ]:
response = qp.run(
    query_str="What is the key of the column outlining survial?",
)

In [ ]:
print(response.message.content)

In [ ]:
response = qp.run(
    query_str="Generate python code to plot survival rate versus fare using matplotlib. Choose an appropriate binsize. Show plot as well as code.",
)

In [ ]:
print(response.message.content)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df is your DataFrame and it contains 'fare' and 'survived' columns
df['fare'] = pd.to_numeric(df['fare'], errors='coerce')  # Ensure fare is numeric
df = df.dropna(subset=['fare', 'survived'])  # Drop rows with missing values

# Group by fare and calculate the mean survival rate
survival_rate_by_fare = df.groupby('fare')['survived'].mean()

# Plot the survival rate versus fare
plt.figure(figsize=(10, 6))
plt.bar(survival_rate_by_fare.index, survival_rate_by_fare.values, color='blue')
plt.xlabel('Fare')
plt.ylabel('Survival Rate')
plt.title('Survival Rate vs Fare')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()